# Digital Waveguides

## 1. Introduction

## 2. Time Domain Digital Waveguide Modeling

A digital waveguide consists of two delay lines representing two travelling waves moving in opposite directions. By summing the values at a certain location along the delay lines at every timestep, it is obtained a waveform. This waveform is the sound heard with the pickup point placed at that relative location. The delay elements are initialized with a shape corresponding to the initial displacement of the string. For simplicity a triangular wave is used even though in reality the initial displacement of a plucked string will not be shaped exactly like a triangle. Simply using two delay lines in this fashion would require arbitrarily long delay lines depending on the length of the desired output. By feeding the delay lines into each other a system can be created that can run for an arbitrary amount of time using fixed size delay elements

<br><br/>
<p align="center"><img src="images/waveguide_initialcond.png" style="width:70%"></p>
<center> Fig. 21$\quad$ Digital waveguide with initial conditions of delay lines set to triangular waves. </center>
<br><br/>
<br><br/>

In modeling a guitar it is important to note that the ends of the string are rigidly terminated, so the waves reflect at either end of the string. This effect can be modelled by negating each sample after it reaches the end of a delay line, before feeding it into the next delay line, as shown in Figure 21. Finally, it is a must to add an attenuation factor. Without the attenuation factor, the model described up until now results in ideal string vibration that never decays. In the real world, due to friction and air resistance, the amplitude of the string vibrations decay over time, so it is important to model this effect in the digital waveguide. To attenuate the output we simply add a damping factor at the ends of the delay lines so that the values are damped before being fed into the other delay line. We tried different values for the damping factor until we found one that we felt did not cause the output to decay too slowly or too quickly. 

<br><br/>
<p align="center"><img src="images/waveguide_rigidterm.png" style="width:70%"></p>
<center> Fig. 22$\quad$ Order N digital waveguide with rigid terminations corresponding to the nut and bridge of a guitar. </center>
<br><br/>
<br><br/>

The length of the delay lines controls the frequency of oscillation, and consequently the pitch of the output signal. This corresponds to fretting a string on a guitar. Fretting a string limits the vibration to a certain length of the string. This changes the wavelength of the travelling waves, which in turn changes the pitch of the sound. Due to the looping nature of waveguide and the lack of additional input the output at every period is the same except attenuated slightly. Therefore the overall output will be periodic with a period depending to the length of the delay line. Therefore, if the desired frequency of the output is f and the sampling frequency is fs, then each delay line length is set to N/2 where N = fs/f.

The sound synthesized by this model sounds very artificial. It does nothing to account for the timbre of the instrument, and modeling the string pluck as a triangle wave is not very accurate. In addition, it does not take into account the fact that a real string vibrates in both the horizontal and vertical planes and interacts with the other strings on the guitar. Despite this, it is important to note that it does get a lot right. The damping of the string depends on the frequency - low pitched notes have a lot of sustain whereas high frequency notes attenuate very rapidly. The digital waveguide model simulates this frequency dependant damping effect quite well. It also does a good job creating audible harmonics present in the sound of any stringed instrument. While the basic digital waveguide plucked string model does a good job simulating a generic plucked string, special considerations for the specific instrument being modelled must also be taken into account.

One problem with implementing this system is that the size of the delay lines must be an integer to create a digital filter. If we wish to always use a set sampling frequency, then the delay line lengths will not always be integers. For example to generate the note A4, if the sampling frequency = 44.1 kHz the delay line length would be 44100/370 = 119.189. In this case if we set the delay line length to 119 the output would be of frequency 44100/119 = 370.588. Thus, the resulting output is slightly out of tune. This effect gets greater as the frequency is increased. Another problem with creating this system in the time domain is that it is computationally expensive [Synthesizing a three second tone takes over one minute].

## 3. Digital Waveguide Implementation Using Digital Filtering Techniques

To simplify the implementation of the waveguide, the two delay lines can be combined into one, and the damping values at the terminations can be lumped together in the feedback loop (See Figure 31). The -1 multipliers cancel each other out, and the two delay lines can be combined leaving only a length N delay line and the damping factors. The damping factors at each delay can then be lumped together into one damping factor

<br><br/>
<p align="center"><img src="images/waveguide_lumpeddelay.png" style="width:50%"></p>
<center> Fig. 31$\quad$  Simplified digital waveguide after combining delay lines and damping factors. </center>
<br><br/>
<br><br/>

Now the model closely resembles the model originally proposed by [Karplus and Strong](K. Karplus and A. Strong, ``Digital synthesis of plucked string and drum timbres,'' Computer Music Journal, vol. 7, no. 2, pp. 43-55, 1983). However, in a real guitar not all frequencies will decay at equal rates. Therefore, for further realism the lumped damping factor is replaced by a 'loop filter' that damps each frequency differently. This loop filter always has a low pass characteristic to it. In the Karplus-Strong model this loop filter is a single zero FIR filter that averages the Nth and N+1th sample. This corresponds to the following difference equation: 

$$
Y[k]=.5 \star(Y[k-N]+Y[k-N-1])
$$

Another difference in the Karplus-Strong model is that white noise is used as the initial conditions. The periodic nature of the filter creates a steady state output that is of the proper frequency regardless of the initial conditions. Using white noise it is very difficult to accurately reproduce the attack portion of a guitar pluck. Another approach that can more accurately synthesize the attack will discused later, It consists of a combination of the pluck sound and the impulse response of the guitar body shape.  It takes into account the effect that the guitar body shape has on the string excitation, resulting in an even more accurate model.

The effect of the guitar body shape on string vibration is more complex than what could be modelled by a low order loop filter as is consider in [Karplus and Strong paper](https://github.com/Xiuhcoatl-013/Numerical-Methods/blob/master/karplus_strong_algorithm/references/Digital Synthesis of Plucked-String and Drum Timbres - K. Karplus, A. Strong [paper].pdf). A more complete model would incorporate the impulse response of the guitar body into the excitation signals and the resonance phenomena due for the body shape. In addition, the strings in any stringed instrument are coupled in complex ways, which the model needs to take into account. Finally, a real string vibrates in both the vertical and horizontal planes, adding additional color to the sound (see [Smith, Julius O. Digital Waveguide Modeling of Musical Instruments, Center for Computer Research in Music and Acoustics (CCRMA), Stanford University, 2003-12-10.](https://github.com/Xiuhcoatl-013/Numerical-Methods/blob/master/digital_waveguides/references/Physical Modeling Using Digital Waveguides - J. O Smith III [paper].pdf)).